In [1]:
import os
import logging
import pandas as pd
import dtale
import numpy as np
from sklearn.feature_selection import chi2, VarianceThreshold, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import eli5
from util import get_dialect
import homophone
import imp

logging.getLogger().setLevel(logging.INFO)
np.random.seed(77213)

In [2]:
prefix = r'D:\git\zhongguoyuyan\csv'
location = pd.read_csv(os.path.join(prefix, 'dialect', 'location.csv'), index_col=0)
char = pd.read_csv(os.path.join(prefix, 'words.csv'), index_col=0)
data = homophone.load_data(os.path.join(prefix, 'dialect'), location.index).reindex(char.index, axis=1, level=1)

INFO:root:loading 1287 data files ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K06mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K10mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K09mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K11mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K07mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06K08mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F98mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F90mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F89mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F96mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F91mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F97mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\csv\dialect\06F87mb01dz.csv ...
INFO:root:loading D:\git\zhongguoyuyan\

In [3]:
train_loc, test_loc, train_data, test_data = train_test_split(location, data, test_size=0.2)
train_dialect = get_dialect(train_loc)
test_dialect = get_dialect(test_loc)

In [4]:
transformer = homophone.ColumnTransformer(transformers=[(
    col,
    homophone.make_pipeline(
        homophone.PhoneClustering(dtype=np.float32, n_clusters=200),
        homophone.Homophone(dtype=np.float32),
        homophone.VarianceThreshold(threshold=0.998 * (1 - 0.998)),
        homophone.HomophoneClustering(n_clusters=1000)
    ),
    train_data.columns.get_loc(col)
) for col in train_data.columns.levels[0]], sparse_threshold=1, n_jobs=3)
train_features = transformer.fit_transform(train_data)
feature_names = np.asarray(transformer.get_feature_names(char['item'].reindex(train_data.columns, level=1)))

In [5]:
idx = np.argsort(-VarianceThreshold(0).fit(train_features).variances_)[:100]
df = pd.DataFrame(train_features[:, idx].A, index=train_data.index, columns=feature_names[idx])
df.insert(0, 'dialect', train_dialect)
dtale.show(df, name='train_features_variance')

In [6]:
selector = SelectKBest(chi2, k=1000).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
idx = np.argsort(-selector.scores_)[:100]
df = pd.DataFrame(
    train_features[np.ix_(train_dialect != '', idx)].A,
    index=train_data[train_dialect != ''].index,
    columns=feature_names[idx]
)
df.insert(0, 'dialect', train_dialect[train_dialect != ''])
dtale.show(df, name='train_features_chi2')

In [7]:
lr = LogisticRegression(penalty='l1', solver='saga', fit_intercept=False).fit(train_features[train_dialect != ''], train_dialect[train_dialect != ''])
test_features = transformer.transform(test_data)
print(classification_report(train_dialect[train_dialect != ''], lr.predict(train_features[train_dialect != ''])))
print(classification_report(test_dialect[test_dialect != ''], lr.predict(test_features[test_dialect != ''])))

              precision    recall  f1-score   support

        东北官话       0.87      1.00      0.93        27
        中原官话       0.97      0.95      0.96        62
        兰银官话       1.00      1.00      1.00        11
        冀鲁官话       0.96      1.00      0.98        23
        北京官话       1.00      1.00      1.00        11
         吴方言       0.98      1.00      0.99        47
        客家方言       0.97      0.97      0.97        34
          平话       0.88      0.78      0.82         9
         徽方言       1.00      1.00      1.00        11
         晋方言       0.97      0.97      0.97        35
        江淮官话       1.00      0.97      0.98        33
        湖南土话       0.00      0.00      0.00         1
         湘方言       0.96      0.93      0.95        28
         粤方言       0.95      0.97      0.96        38
        胶辽官话       1.00      0.71      0.83        14
        西南官话       0.99      1.00      1.00       147
         赣方言       0.98      1.00      0.99        57
         闽方言       0.96    

In [8]:
eli5.explain_weights(lr, top=10, feature_names=feature_names)

Explanation(estimator="LogisticRegression(fit_intercept=False, penalty='l1', solver='saga')", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='东北官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_螺奴吴犁煤楼流岩林难兰颜连莲园银轮匀娘王明浓龙=吉橘竹菊足_爬茶蛇赔皮锤肥桥条头潭蚕钳沉钱前还魂裙糖床黄藤虫穷=吉橘竹菊足_华如余怀时垂严嫌延言完顽权圆原民邻神唇纯坟防能绳行形荣营熊雄容=吉橘竹菊足_吉橘竹菊足=河茄婆斜图壶徐除锄台财排牌鞋回池骑迟祠棋随桃朝浮绸愁球含咸甜寻琴弹缠盘全传船贫陈辰勤盆长尝房狂朋层棚平程城瓶停蓬铜红_吉橘竹菊足=锣鹅牙爷鱼来埋泥雷儿移眉梨姨围毛熬摇牛油南蓝盐棉年人门云忙迎名赢零脓', weight=0.5546144942017147, std=None, value=None), FeatureWeight(feature='tone_螺奴吴犁煤楼流岩林难兰颜连莲园银轮匀娘王明浓龙=叔_叔=柴田_叔=爬茶蛇赔皮锤肥桥条头潭蚕钳沉钱前还魂裙糖床黄藤虫穷_叔=华如余怀时垂严嫌延言完顽权圆原民邻神唇纯坟防能绳行形荣营熊雄容_叔=河茄婆斜图壶徐除锄台财排牌鞋回池骑迟祠棋随桃朝浮绸愁球含咸甜寻琴弹缠盘全传船贫陈辰勤盆长尝房狂朋层棚平程城瓶停蓬铜红_叔=锣鹅牙爷鱼来埋泥雷儿移眉梨姨围毛熬摇牛油南蓝盐棉年人门云忙迎名赢零脓', weight=0.38734898852154714, std=None, value=None), FeatureWeight(feature='tone_低饥关翻姜耕兵终=拍拆_低饥关翻姜耕兵终=吃哭_拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲=拍拆_拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲=吃哭_多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞刀高包抓交焦烧腰箫钩修州休优丢拉三甘衫尖心参针深金音单肝安山间班奸鲜先肩烟搬端酸官欢弯砖冤根恩新身筋墩蹲孙婚温均分军熏帮张装霜章伤秧光慌方桩双江灯僧冰升生更争贞声钉星经兄东公翁冬松风丰中宫封蜂恭凶=拍拆_多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞刀高包抓交焦烧腰箫钩修州休优丢拉三甘衫尖心参针深金音单肝安山间班奸鲜先肩烟搬端酸官欢弯砖冤根恩新身筋墩蹲孙婚温均分军熏帮张装霜章伤秧光慌方桩双江灯僧冰升生更争贞声钉星经兄东公翁冬松风丰中宫封蜂恭凶=吃哭', weight=0.3664517854858483, std=None, value=None), FeatureWeight(feature='finals_歌个可鹅饿河课=破婆磨磨摸', weight=0.3651494835831855, std=None, value=None), FeatureWeight(feature='finals_拨泼末=车蛇射_车蛇射=破婆磨磨摸_折撤舌设热=破婆磨磨摸', weight=0.33732580091929387, std=None, value=None), FeatureWeight(feature='tone_螺奴吴犁煤楼流岩林难兰颜连莲园银轮匀娘王明浓龙=夹_柴田=夹_爬茶蛇赔皮锤肥桥条头潭蚕钳沉钱前还魂裙糖床黄藤虫穷=夹_夹=华如余怀时垂严嫌延言完顽权圆原民邻神唇纯坟防能绳行形荣营熊雄容_夹=河茄婆斜图壶徐除锄台财排牌鞋回池骑迟祠棋随桃朝浮绸愁球含咸甜寻琴弹缠盘全传船贫陈辰勤盆长尝房狂朋层棚平程城瓶停蓬铜红_夹=锣鹅牙爷鱼来埋泥雷儿移眉梨姨围毛熬摇牛油南蓝盐棉年人门云忙迎名赢零脓', weight=0.30123102733402213, std=None, value=None), FeatureWeight(feature='initial_左租做紫资子早灶走卒作粽=纯_坐字贼=纯_纯=嘴醉', weight=0.2720991030282472, std=None, value=None)], neg=[FeatureWeight(feature='tone_七虱一出塞黑织屋=折节结国_吸失息积惜击=折节结国_接跌贴歇切泼脱缺劈踢锡=笔骨谷曲_接跌贴歇切泼脱缺劈踢锡=吉橘竹菊足_接跌贴歇切泼脱缺劈踢锡=北得刻色福宿缩_笔骨谷曲=搭鸭擦割八扎杀瞎刷刮托约郭剥桌_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=吉橘竹菊足_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=北得刻色福宿缩_折节结国=北得刻色福宿缩', weight=-0.3898695695894579, std=None, value=None), FeatureWeight(feature='tone_接跌贴歇切泼脱缺劈踢锡=各恶客策_接跌贴歇切泼脱缺劈踢锡=折节结国_接跌贴歇切泼脱缺劈踢锡=塔甲法_接跌贴歇切泼脱缺劈踢锡=壳格隔_接跌贴歇切泼脱缺劈踢锡=铁雪血_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=各恶客策_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=折节结国_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=壳格隔_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=铁雪血_折节结国=折节结国', weight=-0.37909111556630903, std=None, value=None), FeatureWeight(feature='tone_七虱一出塞黑织屋=笔骨谷曲_七虱一出塞黑织屋=吉橘竹菊足_吸失息积惜击=笔骨谷曲_吸失息积惜击=吉橘竹菊足_吸失息积惜击=北得刻色福宿缩_笔骨谷曲=吉橘竹菊足_搭鸭擦割八扎杀瞎刷刮托约郭剥桌=塔甲法_吉橘竹菊足=北得刻色福宿缩', weight=-0.37543893262837214, std=None, value=None)], pos_remaining=64, neg_remaining=30), proba=None, score=None, weighted_spans=None, heatmap=None), TargetExplanation(target='中原官话', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='tone_低饥关翻姜耕兵终=叶辣热月落药麦_低饥关翻姜耕兵终=日木目_低饥关翻姜耕兵终=业灭列孽末越_低饥关翻姜耕兵终=蜡袜_低饥关翻姜耕兵终=密墨力鹿绿_叶辣热月落药麦=拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲_叶辣热月落药麦=多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞刀高包抓交焦烧腰箫钩修州休优丢拉三甘衫尖心参针深金音单肝安山间班奸鲜先肩烟搬端酸官欢弯砖冤根恩新身筋墩蹲孙婚温均分军熏帮张装霜章伤秧光慌方桩双江灯僧冰升生更争贞声钉星经兄东公翁冬松风丰中宫封蜂恭凶_日木目=拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲_日木目=多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞刀高包抓交焦烧腰箫钩修州休优丢拉三甘衫尖心参针深金音单肝安山间班奸鲜先肩烟搬端酸官欢弯砖冤根恩新身筋墩蹲孙婚温均分军熏帮张装霜章伤秧光慌方桩双江灯僧冰升生更争贞声钉星经兄东公翁冬松风丰中宫封蜂恭凶_拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲=业灭列孽末越_拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲=蜡袜_拖靴车铺箍初区胎开梯希吹亏抄敲偷抽贪签添天牵宽圈吞亲村春汤仓糠疮筐窗坑清轻厅青通葱充冲=密墨力鹿绿_业灭列孽末越=多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞刀高包抓交焦烧腰箫钩修州休优丢拉三甘衫尖心参针深金音单肝安山间班奸鲜先肩烟搬端酸官欢弯砖冤根恩新身筋墩蹲孙婚温均分军熏帮张装霜章伤秧光慌方桩双江灯僧冰升生更争贞声钉星经兄东公翁冬松风丰中宫封蜂恭凶_蜡袜=多歌沙瓜花租乌猪书输该街西鸡溪杯灰歪碑知资师丝衣规追龟飞